In [14]:
%%bash 
echo """
{
  "solver": {
    "lr_policy": "fixed",
    "display": 1000,
    "max_iter":256054,
    "gpu": [0,1,2,3,4,5,6,7],
    "batchsize": 16384,
    "snapshot": 10000000,
    "snapshot_prefix": "./",
    "eval_interval": 5000,
    "eval_batches": 5441,
    "mixed_precision": 1024,
    "eval_metrics": ["AUC:0.8025"]
  },
  
  "optimizer": {
    "type": "SGD",
    "global_update": false,
    "sgd_hparam": {
    "learning_rate": 8.0,
      "warmup_steps": 1000
    }
  },


  "layers": [ 
      {
	 "name": "data",
	  "type": "Data",
	  "format": "Raw",
	  "num_samples": 33711070,
	  "slot_size_array": [138493 , 26744],
	  "source": "./filelist.txt",
	  "eval_num_samples": 276986,
	  "eval_source": "./test_huge_ctr_data.dat",
	  "check": "None",
	  "cache_eval_data": true,
	  "label": {
              "top": "label",
              "label_dim": 1
	  },
      "dense": {
              "top": "dense",
              "dense_dim": 1
	  },
	  "sparse": [
              {
		  "top": "data1",
		  "type": "LocalizedSlot",
		  "max_feature_num_per_sample": 2,
          "max_nnz": 1,
		  "slot_num": 2
              }
	  ]
    },

    {
      "name": "sparse_embedding1",
      "type": "LocalizedSlotSparseEmbeddingOneHot",
      "bottom": "data1",
      "top": "sparse_embedding1",
      "sparse_embedding_hparam": {
        "slot_size_array": [138493 , 26744],
        "embedding_vec_size": 128,
        "combiner": 0
      }
    },

    {
      "name": "fc1",
      "type": "FusedInnerProduct",
      "bottom": "dense",
      "top": "fc1",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc2",
      "type": "FusedInnerProduct",
      "bottom": "fc1",
      "top": "fc2",
      "fc_param": {
        "num_output": 256
      }
    },

   
    {
      "name": "fc3",
      "type": "FusedInnerProduct",
      "bottom": "fc2",
      "top": "fc3",
      "fc_param": {
        "num_output": 128
      }
    },

    {
      "name": "interaction1",
      "type": "Interaction",
      "bottom": ["fc3", "sparse_embedding1"],
      "top": "interaction1"
    },

    {
      "name": "fc4",
      "type": "FusedInnerProduct",
      "bottom": "interaction1",
      "top": "fc4",
       "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc5",
      "type": "FusedInnerProduct",
      "bottom": "fc4",
      "top": "fc5",
      "fc_param": {
        "num_output": 1024
      }
    },

    {
      "name": "fc6",
      "type": "FusedInnerProduct",
      "bottom": "fc5",
      "top": "fc6",
      "fc_param": {
        "num_output": 512
      }
    },

    {
      "name": "fc7",
      "type": "FusedInnerProduct",
      "bottom": "fc6",
      "top": "fc7",
      "fc_param": {
        "num_output": 256
      }
    },

    {
      "name": "fc8",
      "type": "InnerProduct",
      "bottom": "fc7",
      "top": "fc8",
      "fc_param": {
        "num_output": 1
      }
    },
    
    {
      "name": "loss",
      "type": "BinaryCrossEntropyLoss",
      "bottom": ["fc8","label"],
      "top": "loss"
    } 
  ]
}
""" >> movie_lense.json

In [16]:
!cp ../samples/dlrm/dlrm_fp16_16k.json ./movie_lense.json

In [21]:
!../build/bin/huge_ctr --train ./movie_lense.json

[0, init_start, ]
HugeCTR Version: 2.2
Config file: ./movie_lense.json
[09d15h32m39s][HUGECTR][INFO]: batchsize_eval is not specified using default: 16384
Mixed Precision training with scaler: 1024 is enabled.
[09d15h32m39s][HUGECTR][INFO]: algorithm_search is not specified using default: 1
[09d15h32m39s][HUGECTR][INFO]: Algorithm search: ON
[09d15h32m41s][HUGECTR][INFO]: Peer-to-peer access cannot be fully enabled.
Device 0: Tesla V100-SXM2-16GB
[09d15h32m41s][HUGECTR][INFO]: Initial seed is 2617380611
[09d15h32m41s][HUGECTR][INFO]: Vocabulary size: 165237
[09d15h32m41s][HUGECTR][INFO]: num_internal_buffers 1
[09d15h32m41s][HUGECTR][INFO]: num_internal_buffers 17
[09d15h32m41s][HUGECTR][INFO]: max_vocabulary_size_per_gpu_=165237
[09d15h33m01s][HUGECTR][INFO]: gpu0 start to init embedding of slot0 , slot_size=138493, key_offset=0, value_index_offset=0
[09d15h33m01s][HUGECTR][INFO]: gpu0 start to init embedding of slot1 , slot_size=26744, key_offset=138493, value_index_offset=138493
[09

In [20]:
!nvidia-smi

Sun Aug  9 15:16:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.24       Driver Version: 450.24       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   35C    P0    57W / 300W |   1404MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 Off |                    0 |
| N/A   